GridSearch (choose parameter to try, lr, dropout)
early stopping

Deactivating callbacks can be especially useful when you do a parameter search (say with sklearn GridSearchCV). If, for instance, you use a callback for learning rate scheduling (e.g. via LRScheduler) and want to test its usefulness, you can compare the performance once with and once without the callback.

Implement confusion matrix and progress bar

key_ignore=confusion_matrix put in realization

implement early stopping

implement model save

implement gridSearch

implement space_cnn

implement long core for cnn

output labels

了解进行了多少了，直观显示训练过程（训练了多少步骤，百分比，训练误差，训练准确率）


In [1]:
import os
import numpy as np
import scipy.io

In [2]:
mods = ['BPSK', 'DQPSK', 'GFSK', 'GMSK', 'OQPSK',
        'PAM4', 'PAM8', 'PSK8', 'QAM16', 'QAM64', 'QPSK']
class_num = len(mods)

In [3]:
data = scipy.io.loadmat(
    "D:/Archive/0006/"
    "batch100000_symbols128_sps8_baud1_snr5.dat",
)

In [4]:
def import_from_mat(data, size):
    features = []
    labels = []
    for mod in mods:
        real = np.array(data[mod].real[:size])
        imag = np.array(data[mod].imag[:size])
        signal = np.concatenate([real, imag], axis=1)
        features.append(signal)
        labels.append(mods.index(mod) * np.ones([size, 1]))

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return features, labels

In [5]:
features, labels = import_from_mat(data,1000)

In [6]:
features = features.astype(np.float32)
labels = labels.astype(np.int64)

In [7]:
X = features
y = labels.reshape(-1)

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class Discriminator(nn.Module):
    """Define the model"""

    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(2, 256, 3, padding=1),  # batch, 256, 1024
            nn.BatchNorm1d(256),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(256, 80, 3, padding=1),  # batch, 80, 1024
            nn.BatchNorm1d(80),
            nn.ReLU(),
            # nn.Dropout2d()
        )
        self.fc1 = nn.Sequential(
            nn.Linear(80 * 1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=0.6)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(256, class_num),
            nn.ReLU()
        )
        
    def forward(self, x, **kwargs):
        x = x.reshape((x.size(0), 2, -1))
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [10]:
from skorch import NeuralNetClassifier
from skorch.callbacks import EpochScoring, PrintLog, ProgressBar, Callback
from sklearn.metrics import confusion_matrix
from skorch.utils import data_from_dataset
import sys

In [11]:
class Score_ConfusionMatrix(EpochScoring):
    def on_epoch_end(self, net, dataset_train, dataset_valid, **kwargs):
        
        EpochScoring.on_epoch_end(self, net, dataset_train, dataset_valid)
        
        X_test, y_test = data_from_dataset(dataset_valid)
        y_pred = net.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        history = net.history
        history.record("confusion_matrix", cm)

In [45]:
import pdb

class Print_Score_CM(Callback):
    
    def __init__(self, size):
        self.batch_num = 0
        self.sample_num = 0
        self.size = size
        
    def on_batch_end(self, net, **kwargs):
        self.batch_num += 1
        self.sample_num += kwargs['X'].shape[0]
        if self.batch_num % 1 == 0:
            percent = self.sample_num/self.size
            history = net.history
            training = kwargs["training"]
            if training:
                print('Processed: {0:.4f}, ' 
                      'train_batch: {1}, '
                      'train_loss:{2:.4f}'.format(
                          percent,
                          history[-1, "batches", -1, "train_batch_size"],
                          history[-1, "batches", -1, "train_loss"]
                      ))
            else:
                print('Processed: {0:.4f}, ' 
                      'valid_batch: {1}, '
                      'valid_loss:{2:.4f}'.format(
                          percent,
                          history[-1, "batches", -1, "valid_batch_size"],
                          history[-1, "batches", -1, "valid_loss"]
                      ))
    
    def on_epoch_end(self, net, **kwargs):
        self.batch_num = 0
        self.sample_num = 0
        history = net.history
        result = history[-1].copy()
        result.pop("confusion_matrix")
        result.pop("batches")
        print("Epoch: {0}, "
              "Duration: {1:.4f}, "
              "Val_Acc: {2:.4f}, "
              "Acc_best: {3}".format(
                  history[-1, "epoch"],
                  history[-1, "dur"],
                  history[-1, "accuracy"],
                  history[-1, "accuracy_best"]
              ))
        print("Confusion matrix:\n {0}".format(
            history[-1, "confusion_matrix"]
        ))

In [46]:
net = NeuralNetClassifier(
    Discriminator,
    max_epochs=20,
    lr=0.01,
    device='cuda',
    iterator_train__shuffle=True,
    iterator_valid__shuffle=False
)

score = Score_ConfusionMatrix(scoring="accuracy", lower_is_better=False)
pt = Print_Score_CM(X.shape[0])
net.set_params(callbacks__valid_acc=score)
net.set_params(callbacks__print_log=pt)

<class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class '__main__.Discriminator'>,
)

In [47]:
net.fit(X,y)

Processed: 0.0116, train_batch: 128, train_loss:2.4199
Processed: 0.0233, train_batch: 128, train_loss:2.2424
Processed: 0.0349, train_batch: 128, train_loss:2.0653
Processed: 0.0465, train_batch: 128, train_loss:1.9768
Processed: 0.0582, train_batch: 128, train_loss:1.9863
Processed: 0.0698, train_batch: 128, train_loss:1.9302
Processed: 0.0815, train_batch: 128, train_loss:1.8163
Processed: 0.0931, train_batch: 128, train_loss:1.8276
Processed: 0.1047, train_batch: 128, train_loss:1.9158
Processed: 0.1164, train_batch: 128, train_loss:1.7253
Processed: 0.1280, train_batch: 128, train_loss:1.7807
Processed: 0.1396, train_batch: 128, train_loss:1.7562
Processed: 0.1513, train_batch: 128, train_loss:1.6502
Processed: 0.1629, train_batch: 128, train_loss:1.7317
Processed: 0.1745, train_batch: 128, train_loss:1.6132
Processed: 0.1862, train_batch: 128, train_loss:1.7264
Processed: 0.1978, train_batch: 128, train_loss:1.6588
Processed: 0.2095, train_batch: 128, train_loss:1.5806
Processed:

Processed: 0.6051, train_batch: 128, train_loss:1.0378
Processed: 0.6167, train_batch: 128, train_loss:0.9304
Processed: 0.6284, train_batch: 128, train_loss:0.9383
Processed: 0.6400, train_batch: 128, train_loss:0.9506
Processed: 0.6516, train_batch: 128, train_loss:0.8956
Processed: 0.6633, train_batch: 128, train_loss:0.9720
Processed: 0.6749, train_batch: 128, train_loss:0.9109
Processed: 0.6865, train_batch: 128, train_loss:0.8022
Processed: 0.6982, train_batch: 128, train_loss:0.9388
Processed: 0.7098, train_batch: 128, train_loss:0.8861
Processed: 0.7215, train_batch: 128, train_loss:0.9196
Processed: 0.7331, train_batch: 128, train_loss:0.8730
Processed: 0.7447, train_batch: 128, train_loss:0.9012
Processed: 0.7564, train_batch: 128, train_loss:0.9800
Processed: 0.7680, train_batch: 128, train_loss:0.8389
Processed: 0.7796, train_batch: 128, train_loss:0.9752
Processed: 0.7913, train_batch: 128, train_loss:0.9528
Processed: 0.8000, train_batch: 96, train_loss:0.9647
Processed: 

Processed: 0.0582, train_batch: 128, train_loss:0.6275
Processed: 0.0698, train_batch: 128, train_loss:0.6773
Processed: 0.0815, train_batch: 128, train_loss:0.7904
Processed: 0.0931, train_batch: 128, train_loss:0.6856
Processed: 0.1047, train_batch: 128, train_loss:0.6524
Processed: 0.1164, train_batch: 128, train_loss:0.6096
Processed: 0.1280, train_batch: 128, train_loss:0.7700
Processed: 0.1396, train_batch: 128, train_loss:0.6469
Processed: 0.1513, train_batch: 128, train_loss:0.7420
Processed: 0.1629, train_batch: 128, train_loss:0.7101
Processed: 0.1745, train_batch: 128, train_loss:0.6759
Processed: 0.1862, train_batch: 128, train_loss:0.6682
Processed: 0.1978, train_batch: 128, train_loss:0.5567
Processed: 0.2095, train_batch: 128, train_loss:0.6765
Processed: 0.2211, train_batch: 128, train_loss:0.5904
Processed: 0.2327, train_batch: 128, train_loss:0.6039
Processed: 0.2444, train_batch: 128, train_loss:0.6302
Processed: 0.2560, train_batch: 128, train_loss:0.6939
Processed:

Processed: 0.6516, train_batch: 128, train_loss:0.5911
Processed: 0.6633, train_batch: 128, train_loss:0.4928
Processed: 0.6749, train_batch: 128, train_loss:0.4691
Processed: 0.6865, train_batch: 128, train_loss:0.5429
Processed: 0.6982, train_batch: 128, train_loss:0.5340
Processed: 0.7098, train_batch: 128, train_loss:0.4987
Processed: 0.7215, train_batch: 128, train_loss:0.4924
Processed: 0.7331, train_batch: 128, train_loss:0.5778
Processed: 0.7447, train_batch: 128, train_loss:0.5345
Processed: 0.7564, train_batch: 128, train_loss:0.5087
Processed: 0.7680, train_batch: 128, train_loss:0.5188
Processed: 0.7796, train_batch: 128, train_loss:0.5029
Processed: 0.7913, train_batch: 128, train_loss:0.5109
Processed: 0.8000, train_batch: 96, train_loss:0.6310
Processed: 0.8116, valid_batch: 128, valid_loss:0.0856
Processed: 0.8233, valid_batch: 128, valid_loss:0.7775
Processed: 0.8349, valid_batch: 128, valid_loss:1.6628
Processed: 0.8465, valid_batch: 128, valid_loss:0.2678
Processed: 

Processed: 0.1047, train_batch: 128, train_loss:0.3786
Processed: 0.1164, train_batch: 128, train_loss:0.4508
Processed: 0.1280, train_batch: 128, train_loss:0.3910
Processed: 0.1396, train_batch: 128, train_loss:0.3897
Processed: 0.1513, train_batch: 128, train_loss:0.3965
Processed: 0.1629, train_batch: 128, train_loss:0.5113
Processed: 0.1745, train_batch: 128, train_loss:0.4215
Processed: 0.1862, train_batch: 128, train_loss:0.3871
Processed: 0.1978, train_batch: 128, train_loss:0.4684
Processed: 0.2095, train_batch: 128, train_loss:0.4293
Processed: 0.2211, train_batch: 128, train_loss:0.4082
Processed: 0.2327, train_batch: 128, train_loss:0.3826
Processed: 0.2444, train_batch: 128, train_loss:0.3852
Processed: 0.2560, train_batch: 128, train_loss:0.3722
Processed: 0.2676, train_batch: 128, train_loss:0.4203
Processed: 0.2793, train_batch: 128, train_loss:0.3779
Processed: 0.2909, train_batch: 128, train_loss:0.4172
Processed: 0.3025, train_batch: 128, train_loss:0.3340
Processed:

...

Processed: 0.1513, train_batch: 128, train_loss:0.2412
Processed: 0.1629, train_batch: 128, train_loss:0.3339
Processed: 0.1745, train_batch: 128, train_loss:0.2280
Processed: 0.1862, train_batch: 128, train_loss:0.2011
Processed: 0.1978, train_batch: 128, train_loss:0.2951
Processed: 0.2095, train_batch: 128, train_loss:0.2685
Processed: 0.2211, train_batch: 128, train_loss:0.2678
Processed: 0.2327, train_batch: 128, train_loss:0.2313
Processed: 0.2444, train_batch: 128, train_loss:0.2289
Processed: 0.2560, train_batch: 128, train_loss:0.2084
Processed: 0.2676, train_batch: 128, train_loss:0.1903
Processed: 0.2793, train_batch: 128, train_loss:0.2894
Processed: 0.2909, train_batch: 128, train_loss:0.3257
Processed: 0.3025, train_batch: 128, train_loss:0.2589
Processed: 0.3142, train_batch: 128, train_loss:0.2554
Processed: 0.3258, train_batch: 128, train_loss:0.2151
Processed: 0.3375, train_batch: 128, train_loss:0.2200
Processed: 0.3491, train_batch: 128, train_loss:0.2493
Processed:

Processed: 0.7447, train_batch: 128, train_loss:0.2497
Processed: 0.7564, train_batch: 128, train_loss:0.1790
Processed: 0.7680, train_batch: 128, train_loss:0.1743
Processed: 0.7796, train_batch: 128, train_loss:0.1786
Processed: 0.7913, train_batch: 128, train_loss:0.2346
Processed: 0.8000, train_batch: 96, train_loss:0.2251
Processed: 0.8116, valid_batch: 128, valid_loss:0.0291
Processed: 0.8233, valid_batch: 128, valid_loss:0.4996
Processed: 0.8349, valid_batch: 128, valid_loss:1.1055
Processed: 0.8465, valid_batch: 128, valid_loss:0.1572
Processed: 0.8582, valid_batch: 128, valid_loss:0.0331
Processed: 0.8698, valid_batch: 128, valid_loss:0.0389
Processed: 0.8815, valid_batch: 128, valid_loss:0.4540
Processed: 0.8931, valid_batch: 128, valid_loss:0.4037
Processed: 0.9047, valid_batch: 128, valid_loss:0.0434
Processed: 0.9164, valid_batch: 128, valid_loss:0.0326
Processed: 0.9280, valid_batch: 128, valid_loss:0.1602
Processed: 0.9396, valid_batch: 128, valid_loss:1.8158
Processed: 

Processed: 0.1978, train_batch: 128, train_loss:0.1541
Processed: 0.2095, train_batch: 128, train_loss:0.1540
Processed: 0.2211, train_batch: 128, train_loss:0.1200
Processed: 0.2327, train_batch: 128, train_loss:0.1564
Processed: 0.2444, train_batch: 128, train_loss:0.1344
Processed: 0.2560, train_batch: 128, train_loss:0.1243
Processed: 0.2676, train_batch: 128, train_loss:0.1322
Processed: 0.2793, train_batch: 128, train_loss:0.1611
Processed: 0.2909, train_batch: 128, train_loss:0.1603
Processed: 0.3025, train_batch: 128, train_loss:0.1446
Processed: 0.3142, train_batch: 128, train_loss:0.1298
Processed: 0.3258, train_batch: 128, train_loss:0.1438
Processed: 0.3375, train_batch: 128, train_loss:0.1309
Processed: 0.3491, train_batch: 128, train_loss:0.1366
Processed: 0.3607, train_batch: 128, train_loss:0.1587
Processed: 0.3724, train_batch: 128, train_loss:0.1967
Processed: 0.3840, train_batch: 128, train_loss:0.1404
Processed: 0.3956, train_batch: 128, train_loss:0.1532
Processed:

Processed: 0.7913, train_batch: 128, train_loss:0.1284
Processed: 0.8000, train_batch: 96, train_loss:0.1430
Processed: 0.8116, valid_batch: 128, valid_loss:0.0419
Processed: 0.8233, valid_batch: 128, valid_loss:0.5906
Processed: 0.8349, valid_batch: 128, valid_loss:1.2510
Processed: 0.8465, valid_batch: 128, valid_loss:0.1544
Processed: 0.8582, valid_batch: 128, valid_loss:0.0229
Processed: 0.8698, valid_batch: 128, valid_loss:0.0207
Processed: 0.8815, valid_batch: 128, valid_loss:0.2659
Processed: 0.8931, valid_batch: 128, valid_loss:0.2359
Processed: 0.9047, valid_batch: 128, valid_loss:0.0210
Processed: 0.9164, valid_batch: 128, valid_loss:0.0228
Processed: 0.9280, valid_batch: 128, valid_loss:0.1297
Processed: 0.9396, valid_batch: 128, valid_loss:1.3001
Processed: 0.9513, valid_batch: 128, valid_loss:0.6613
Processed: 0.9629, valid_batch: 128, valid_loss:0.0352
Processed: 0.9745, valid_batch: 128, valid_loss:0.0274
Processed: 0.9862, valid_batch: 128, valid_loss:0.7727
Processed: 

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Discriminator(
    (conv1): Sequential(
      (0): Conv1d(2, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv1d(256, 80, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (fc1): Sequential(
      (0): Linear(in_features=81920, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.6)
    )
    (fc2): Sequential(
      (0): Linear(in_features=256, out_features=11, bias=True)
      (1): ReLU()
    )
  ),
)